In [ ]:
%pip install llama-index-llms-openai
!pip install llama-index
#import nest_asyncio

#nest_asyncio.apply()
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.ERROR)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))


In [ ]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=['../data/paul_graham_essay3.txt']).load_data()
#documents = SimpleDirectoryReader(input_files=['../data/2022 Q3 AAPL.pdf']).load_data()



In [ ]:
from llama_index.core.node_parser import SentenceSplitter

nodes = SentenceSplitter().get_nodes_from_documents(documents)


In [ ]:
nodes

In [ ]:
from llama_index.core.storage.docstore import SimpleDocumentStore

docstore = SimpleDocumentStore()
docstore.add_documents(nodes)


In [ ]:
len(docstore.docs)


In [ ]:
docstore.docs.keys()


In [ ]:
doc_key = list(docstore.docs.keys())[0]  # Select the first document key
docstore.docs[doc_key]


In [ ]:
node = docstore.docs[doc_key]
print(node.text)  # Displays the text content of the node


In [ ]:
print(node.extra_info)


In [ ]:
for key, document in docstore.docs.items():
    print(f"Key: {key}, Metadata: {document.extra_info}")


In [ ]:
for key, document in docstore.docs.items():
    print(f"Key: {key}")
    print(f"Text: {document.text[:100]}...")  # Display the first 100 characters
    print(f"Metadata: {document.extra_info}")
    print("------------")


In [ ]:
"dbf00ed1-e3c1-4110-9da6-e59a46451672" in docstore.docs


In [ ]:
all_texts = [doc.text for doc in docstore.docs.values()]
print(all_texts)


In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("open ai api key: ")

In [ ]:
from llama_index.core import StorageContext, SummaryIndex, VectorStoreIndex, SimpleKeywordTableIndex

storage_context = StorageContext.from_defaults(docstore=docstore)
summary_index = SummaryIndex(nodes, storage_context=storage_context)
vector_index = VectorStoreIndex(nodes, storage_context=storage_context)
keyword_table_index = SimpleKeywordTableIndex(nodes, storage_context=storage_context)


In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
Settings.llm = llm
Settings.chunk_size = 1024


In [ ]:
query_engine = summary_index.as_query_engine()
response = query_engine.query("What is a summary of this document?")
print(response)




In [ ]:
query_engine = vector_index.as_query_engine()
response = query_engine.query("What did the author do growing up?")
print(response)


In [ ]:
query_engine = keyword_table_index.as_query_engine()
response = query_engine.query("What did the author do after his time at YC?")

In [ ]:
print(response)
